In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from PIL import Image, ImageOps
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

import tensorflow as tf

In [ ]:
saved_model_age = load_model('../../models/model_age.model', compile=True)
saved_model_gen = load_model('../../models/model_gen.model', compile=True)
saved_model_race = load_model('../../models/model_race.model', compile=True)

# saved_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# saved_model.predict([x_test])

# age_model=load_model('../../models/gen_model.h5')
# age_model.summary()

In [ ]:
def process_and_predict(file):
    im = Image.open(file)
    width, height = im.size
    if width == height:
        im = im.resize((96,96))
    else:
        if width > height:
            left = width/2 - height/2
            right = width/2 + height/2
            top = 0
            bottom = height
            im = im.crop((left,top,right,bottom))
            im = im.resize((96,96))
        else:
            left = 0
            right = width
            top = 0
            bottom = width
            im = im.crop((left,top,right,bottom))
            im = im.resize((96,96))
            
    ar = np.asarray(im)
    ar = ar.astype('float32')
    ar /= 255.0
    ar = ar.reshape(-1, 96, 96, 3)
    
    age = int(saved_model_age.predict(ar))
    gender = np.round(saved_model_gen.predict(ar))
    if gender == 0:
        gender = 'male'
    elif gender == 1:
        gender = 'female'
        
    race = np.round(saved_model_race.predict(ar))
    if race == 0:
        race = 'white'
    elif race == 1:
        race = 'black'
    elif race == 2:
        race = 'asian'
    elif race == 3:
        race = 'indian'
    elif race == 4:
        race = 'others'

    print('Age:', age, '\nGender:', gender, '\nRace:', race)
    return im.resize((96,96))

In [ ]:
process_and_predict('/home/vorkov/Documents/x.jpg')